In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [20]:
SHIPS_ROUTES_DATASET_EXTENDED = r"C:\Users\yairs\OneDrive\Documents\University\Master\Ships\datasets from Doron\ships routes\ships routes (ports only) with locations of ports - extended.csv"
OUTPUT_PROCESSED_ROUTES_DATASET = r"C:\Users\yairs\OneDrive\Documents\University\Master\Ships\datasets from Doron\ships routes\extended_proceesed.csv"

routes_extended_df = pd.read_csv(SHIPS_ROUTES_DATASET_EXTENDED)

# Convert columns to right data types
routes_extended_df['Arrival (LT)'] = pd.to_datetime(routes_extended_df['Arrival (LT)'], errors='coerce', dayfirst=True)
routes_extended_df['Departure (LT)'] = pd.to_datetime(routes_extended_df['Departure (LT)'], errors='coerce', dayfirst=True)
routes_extended_df['Longitude'] = pd.to_numeric(routes_extended_df['Longitude'], errors='coerce')
routes_extended_df['Latitude'] = pd.to_numeric(routes_extended_df['Latitude'], errors='coerce')
routes_extended_df['Hours'] = pd.to_numeric(routes_extended_df['Hours'], errors='coerce')


In [21]:
mean_time_in_port_by_ship = routes_extended_df.groupby('Ship').mean()['Hours']

C:\Users\yairs\AppData\Local\Temp\ipykernel_13856\243279495.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  mean_time_in_port_by_ship = routes_extended_df.groupby('Ship').mean()['Hours']


In [22]:
final_ship_dfs = []
for name, ship_df in routes_extended_df.groupby('Ship'):
    # Impute missing Hours (time in port) values
    ship_df.loc[ship_df['Hours'].isna(), 'Hours'] = mean_time_in_port_by_ship[name]
    ship_df['Hours (time delta)'] = pd.to_timedelta(ship_df['Hours'].astype(str) + 'h', errors='coerce')
    
    # Impute missing Departure and Arrival values based on the imputed Hours values
    ship_df.loc[ship_df['Departure (LT)'].isna(), 'Departure (LT)'] = ship_df['Arrival (LT)'] + ship_df['Hours (time delta)']
    ship_df.loc[ship_df['Arrival (LT)'].isna(), 'Arrival (LT)'] = ship_df['Departure (LT)'] - ship_df['Hours (time delta)']
    
    # Prepare final route of the ship
    duplicated_ship_df = ship_df.copy()
    ship_df = ship_df[['Ship', 'Port', 'Arrival (LT)']]
    new_ship_df = ship_df.rename(columns={'Arrival (LT)': 'time'})
    duplicated_ship_df = duplicated_ship_df[['Ship', 'Port', 'Departure (LT)']]
    duplicated_ship_df.rename(columns={'Departure (LT)': 'time'}, inplace=True)
    final_ship_df = pd.concat([new_ship_df, duplicated_ship_df], ignore_index=True)
    final_ship_df.sort_values('time', ascending=True, inplace=True)

    # Convert (absolute) time column to relative time (in hours)
    reference_time_value = final_ship_df.iloc[0]['time']
    relative_times_in_hours = []
    for index, row in final_ship_df.iterrows():
        relative_times_in_hours.append((row['time'] - reference_time_value) / np.timedelta64(1, 'h'))
    final_ship_df['relative time'] = relative_times_in_hours
    final_ship_dfs.append(final_ship_df)

final_ships_df = pd.concat(final_ship_dfs, ignore_index=True)
final_ships_df.to_csv(OUTPUT_PROCESSED_ROUTES_DATASET, index=False, date_format="%d/%m/%y %H:%M:%S")
print(f"There are {len(final_ships_df.groupby('Ship'))} ships in the routes dataset, with a mean route length of {final_ships_df.groupby('Ship').count()['Port'].mean() / 2} ports")

There are 192 ships in the routes dataset, with a mean route length of 63.572916666666664 ports


In [23]:
ports_in_routes_df = routes_extended_df[['Port', 'Longitude', 'Latitude']]
ports_in_routes_df = ports_in_routes_df.drop_duplicates()
ports_in_routes_df['Port'] = ports_in_routes_df['Port'].str.lower()

print(f'There are {len(ports_in_routes_df)} ports in the routes dataset')

There are 115 ports in the routes dataset


In [16]:
TEMPRATURE_2019_JANUARY_PATH = r"C:\Users\yairs\OneDrive\Documents\University\Master\Ships\datasets from Doron\environment conditions\temprature data from NASA earth observations\MYD28M_2019-01-01_rgb_3600x1800.SS.CSV"
TEMPRATURE_2019_APRIL_PATH = r"C:\Users\yairs\OneDrive\Documents\University\Master\Ships\datasets from Doron\environment conditions\temprature data from NASA earth observations\MYD28M_2019-04-01_rgb_3600x1800.SS.CSV"
TEMPRATURE_2019_JULY_PATH = r"C:\Users\yairs\OneDrive\Documents\University\Master\Ships\datasets from Doron\environment conditions\temprature data from NASA earth observations\MYD28M_2019-07-01_rgb_3600x1800.SS.CSV"
TEMPRATURE_2019_OCTOBER_PATH = r"C:\Users\yairs\OneDrive\Documents\University\Master\Ships\datasets from Doron\environment conditions\temprature data from NASA earth observations\MYD28M_2019-10-01_rgb_3600x1800.SS.CSV"

TEMPRATURE_2020_JANUARY_PATH = r"C:\Users\yairs\OneDrive\Documents\University\Master\Ships\datasets from Doron\environment conditions\temprature data from NASA earth observations\MYD28M_2020-01-01_rgb_3600x1800.SS.CSV"
TEMPRATURE_2020_APRIL_PATH = r"C:\Users\yairs\OneDrive\Documents\University\Master\Ships\datasets from Doron\environment conditions\temprature data from NASA earth observations\MYD28M_2020-04-01_rgb_3600x1800.SS.CSV"
TEMPRATURE_2020_JULY_PATH = r"C:\Users\yairs\OneDrive\Documents\University\Master\Ships\datasets from Doron\environment conditions\temprature data from NASA earth observations\MYD28M_2020-07-01_rgb_3600x1800.SS.CSV"
TEMPRATURE_2020_OCTOBER_PATH = r"C:\Users\yairs\OneDrive\Documents\University\Master\Ships\datasets from Doron\environment conditions\temprature data from NASA earth observations\MYD28M_2020-10-01_rgb_3600x1800.SS.CSV"

TEMPRATURE_2021_JANUARY_PATH = r"C:\Users\yairs\OneDrive\Documents\University\Master\Ships\datasets from Doron\environment conditions\temprature data from NASA earth observations\MYD28M_2021-01-01_rgb_3600x1800.SS.CSV"
TEMPRATURE_2021_APRIL_PATH = r"C:\Users\yairs\OneDrive\Documents\University\Master\Ships\datasets from Doron\environment conditions\temprature data from NASA earth observations\MYD28M_2021-04-01_rgb_3600x1800.SS.CSV"
TEMPRATURE_2021_JULY_PATH = r"C:\Users\yairs\OneDrive\Documents\University\Master\Ships\datasets from Doron\environment conditions\temprature data from NASA earth observations\MYD28M_2021-07-01_rgb_3600x1800.SS.CSV"
TEMPRATURE_2021_OCTOBER_PATH = r"C:\Users\yairs\OneDrive\Documents\University\Master\Ships\datasets from Doron\environment conditions\temprature data from NASA earth observations\MYD28M_2021-10-01_rgb_3600x1800.SS.CSV"

TEMPRATURE_2022_JANUARY_PATH = r"C:\Users\yairs\OneDrive\Documents\University\Master\Ships\datasets from Doron\environment conditions\temprature data from NASA earth observations\MYD28M_2022-01-01_rgb_3600x1800.SS.CSV"
TEMPRATURE_2022_APRIL_PATH = r"C:\Users\yairs\OneDrive\Documents\University\Master\Ships\datasets from Doron\environment conditions\temprature data from NASA earth observations\MYD28M_2022-04-15_rgb_3600x1800.SS.CSV"
TEMPRATURE_2022_JULY_PATH = r"C:\Users\yairs\OneDrive\Documents\University\Master\Ships\datasets from Doron\environment conditions\temprature data from NASA earth observations\MYD28M_2022-07-01_rgb_3600x1800.SS.CSV"
TEMPRATURE_2022_OCTOBER_PATH = r"C:\Users\yairs\OneDrive\Documents\University\Master\Ships\datasets from Doron\environment conditions\temprature data from NASA earth observations\MYD28M_2022-10-01_rgb_3600x1800.SS.CSV"

CHLOROPHYL_2019_JANUARY_PATH = r"C:\Users\yairs\OneDrive\Documents\University\Master\Ships\datasets from Doron\environment conditions\chlorophyl data from NASA earth observations\MY1DMM_CHLORA_2019-01-01_rgb_3600x1800.SS.CSV"
CHLOROPHYL_2019_APRIL_PATH = r"C:\Users\yairs\OneDrive\Documents\University\Master\Ships\datasets from Doron\environment conditions\chlorophyl data from NASA earth observations\MY1DMM_CHLORA_2019-04-01_rgb_3600x1800.SS.CSV"
CHLOROPHYL_2019_JULY_PATH = r"C:\Users\yairs\OneDrive\Documents\University\Master\Ships\datasets from Doron\environment conditions\chlorophyl data from NASA earth observations\MY1DMM_CHLORA_2019-07-01_rgb_3600x1800.SS.CSV"
CHLOROPHYL_2019_OCTOBER_PATH = r"C:\Users\yairs\OneDrive\Documents\University\Master\Ships\datasets from Doron\environment conditions\chlorophyl data from NASA earth observations\MY1DMM_CHLORA_2019-10-01_rgb_3600x1800.SS.CSV"

CHLOROPHYL_2020_JANUARY_PATH = r"C:\Users\yairs\OneDrive\Documents\University\Master\Ships\datasets from Doron\environment conditions\chlorophyl data from NASA earth observations\MY1DMM_CHLORA_2020-01-01_rgb_3600x1800.SS.CSV"
CHLOROPHYL_2020_APRIL_PATH = r"C:\Users\yairs\OneDrive\Documents\University\Master\Ships\datasets from Doron\environment conditions\chlorophyl data from NASA earth observations\MY1DMM_CHLORA_2020-04-01_rgb_3600x1800.SS.CSV"
CHLOROPHYL_2020_JULY_PATH = r"C:\Users\yairs\OneDrive\Documents\University\Master\Ships\datasets from Doron\environment conditions\chlorophyl data from NASA earth observations\MY1DMM_CHLORA_2020-07-01_rgb_3600x1800.SS.CSV"
CHLOROPHYL_2020_OCTOBER_PATH = r"C:\Users\yairs\OneDrive\Documents\University\Master\Ships\datasets from Doron\environment conditions\chlorophyl data from NASA earth observations\MY1DMM_CHLORA_2020-10-01_rgb_3600x1800.SS.CSV"

CHLOROPHYL_2021_JANUARY_PATH = r"C:\Users\yairs\OneDrive\Documents\University\Master\Ships\datasets from Doron\environment conditions\chlorophyl data from NASA earth observations\MY1DMM_CHLORA_2021-01-01_rgb_3600x1800.SS.CSV"
CHLOROPHYL_2021_APRIL_PATH = r"C:\Users\yairs\OneDrive\Documents\University\Master\Ships\datasets from Doron\environment conditions\chlorophyl data from NASA earth observations\MY1DMM_CHLORA_2021-04-01_rgb_3600x1800.SS.CSV"
CHLOROPHYL_2021_JULY_PATH = r"C:\Users\yairs\OneDrive\Documents\University\Master\Ships\datasets from Doron\environment conditions\chlorophyl data from NASA earth observations\MY1DMM_CHLORA_2021-07-01_rgb_3600x1800.SS.CSV"
CHLOROPHYL_2021_OCTOBER_PATH = r"C:\Users\yairs\OneDrive\Documents\University\Master\Ships\datasets from Doron\environment conditions\chlorophyl data from NASA earth observations\MY1DMM_CHLORA_2021-10-01_rgb_3600x1800.SS.CSV"

CHLOROPHYL_2022_JANUARY_PATH = r"C:\Users\yairs\OneDrive\Documents\University\Master\Ships\datasets from Doron\environment conditions\chlorophyl data from NASA earth observations\MY1DMM_CHLORA_2022-01-01_rgb_3600x1800.SS.CSV"
CHLOROPHYL_2022_APRIL_PATH = r"C:\Users\yairs\OneDrive\Documents\University\Master\Ships\datasets from Doron\environment conditions\chlorophyl data from NASA earth observations\MY1DMM_CHLORA_2022-04-15_rgb_3600x1800.SS.CSV"
CHLOROPHYL_2022_JULY_PATH = r"C:\Users\yairs\OneDrive\Documents\University\Master\Ships\datasets from Doron\environment conditions\chlorophyl data from NASA earth observations\MY1DMM_CHLORA_2022-07-01_rgb_3600x1800.SS.CSV"

TEMPRATURE_JANUARY_PATHS = [TEMPRATURE_2019_JANUARY_PATH, TEMPRATURE_2020_JANUARY_PATH, TEMPRATURE_2021_JANUARY_PATH, TEMPRATURE_2022_JANUARY_PATH]
TEMPRATURE_APRIL_PATHS = [TEMPRATURE_2019_APRIL_PATH, TEMPRATURE_2020_APRIL_PATH, TEMPRATURE_2021_APRIL_PATH, TEMPRATURE_2022_APRIL_PATH]
TEMPRATURE_JULY_PATHS = [TEMPRATURE_2019_JULY_PATH, TEMPRATURE_2020_JULY_PATH, TEMPRATURE_2021_JULY_PATH, TEMPRATURE_2022_JULY_PATH]
TEMPRATURE_OCTOBER_PATHS = [TEMPRATURE_2019_OCTOBER_PATH, TEMPRATURE_2020_OCTOBER_PATH, TEMPRATURE_2021_OCTOBER_PATH, TEMPRATURE_2022_OCTOBER_PATH]

CHLOROPHYL_JANUARY_PATHS = [CHLOROPHYL_2019_JANUARY_PATH, CHLOROPHYL_2020_JANUARY_PATH, CHLOROPHYL_2021_JANUARY_PATH, CHLOROPHYL_2022_JANUARY_PATH]
CHLOROPHYL_APRIL_PATHS = [CHLOROPHYL_2019_APRIL_PATH, CHLOROPHYL_2020_APRIL_PATH, CHLOROPHYL_2021_APRIL_PATH, CHLOROPHYL_2022_APRIL_PATH]
CHLOROPHYL_JULY_PATHS = [CHLOROPHYL_2019_JULY_PATH, CHLOROPHYL_2020_JULY_PATH, CHLOROPHYL_2021_JULY_PATH, CHLOROPHYL_2022_JULY_PATH]
CHLOROPHYL_OCTOBER_PATHS = [CHLOROPHYL_2019_OCTOBER_PATH, CHLOROPHYL_2020_OCTOBER_PATH, CHLOROPHYL_2021_OCTOBER_PATH]

# Extract long and lat keys from one dataset for example
temprature_2022_january_df = pd.read_csv(TEMPRATURE_2022_JANUARY_PATH)
lat_keys = temprature_2022_january_df['lat/lon'].to_numpy(dtype=float).round(2)
long_keys = temprature_2022_january_df.columns[1:].to_numpy(dtype=float).round(2)

In [14]:
def prepare_temprature_or_chlorophyl_dataset(dataset_path):
    df = pd.read_csv(dataset_path)
    df.columns = ['lat'] + list(long_keys)
    df['lat'] = lat_keys
    return df

In [17]:
temprature_january_dfs = []
for path in TEMPRATURE_JANUARY_PATHS:
    temprature_january_dfs.append(prepare_temprature_or_chlorophyl_dataset(path))

temprature_april_dfs = []
for path in TEMPRATURE_APRIL_PATHS:
    temprature_april_dfs.append(prepare_temprature_or_chlorophyl_dataset(path))

temprature_july_dfs = []
for path in TEMPRATURE_JULY_PATHS:
    temprature_july_dfs.append(prepare_temprature_or_chlorophyl_dataset(path))

temprature_october_dfs = []
for path in TEMPRATURE_OCTOBER_PATHS:
    temprature_october_dfs.append(prepare_temprature_or_chlorophyl_dataset(path))

chlorophyl_january_dfs = []
for path in CHLOROPHYL_JANUARY_PATHS:
    chlorophyl_january_dfs.append(prepare_temprature_or_chlorophyl_dataset(path))

chlorophyl_april_dfs = []
for path in CHLOROPHYL_APRIL_PATHS:
    chlorophyl_april_dfs.append(prepare_temprature_or_chlorophyl_dataset(path))

chlorophyl_july_dfs = []
for path in CHLOROPHYL_JULY_PATHS:
    chlorophyl_july_dfs.append(prepare_temprature_or_chlorophyl_dataset(path))

chlorophyl_october_dfs = []
for path in CHLOROPHYL_OCTOBER_PATHS:
    chlorophyl_october_dfs.append(prepare_temprature_or_chlorophyl_dataset(path))

In [18]:
def get_temprature_or_chlorophyl_value(datasets, lat, long):
    lat_index = datasets[0].index[datasets[0]['lat'] == lat]
    long_index = datasets[0].columns.get_loc(long)
    coordinates_to_check = [(lat_index, long_index), 
                            (lat_index - 1, long_index), (lat_index + 1, long_index),
                            (lat_index, long_index - 1), (lat_index, long_index + 1), 
                            (lat_index - 1, long_index - 1), (lat_index - 1, long_index + 1),
                            (lat_index + 1, long_index - 1), (lat_index + 1, long_index + 1)]
    for lat_index, long_index in coordinates_to_check:
        for dataset in datasets:
            value = dataset.iloc[lat_index, long_index].item()
            if value != 99999:
                return value
    return None

In [25]:
ports_temprature_january = []
ports_temprature_april = []
ports_temprature_july = []
ports_temprature_october = []

ports_chlorophyl_january = []
ports_chlorophyl_april = []
ports_chlorophyl_july = []
ports_chlorophyl_october = []

for index, row in ports_in_routes_df.iterrows():
    port_lat = row['Latitude']
    port_long = row['Longitude']
    closest_valid_lat_key = min(lat_keys, key=lambda x: abs(x - port_lat))
    closest_valid_long_key = min(long_keys, key=lambda x: abs(x - port_long))

    port_temprature_january = get_temprature_or_chlorophyl_value(temprature_january_dfs, closest_valid_lat_key, closest_valid_long_key)
    ports_temprature_january.append(port_temprature_january)
    port_temprature_april = get_temprature_or_chlorophyl_value(temprature_april_dfs, closest_valid_lat_key, closest_valid_long_key)
    ports_temprature_april.append(port_temprature_april)
    port_temprature_july = get_temprature_or_chlorophyl_value(temprature_july_dfs, closest_valid_lat_key, closest_valid_long_key)
    ports_temprature_july.append(port_temprature_july)
    port_temprature_october = get_temprature_or_chlorophyl_value(temprature_october_dfs, closest_valid_lat_key, closest_valid_long_key)
    ports_temprature_october.append(port_temprature_october)

    port_chlorophyl_january = get_temprature_or_chlorophyl_value(chlorophyl_january_dfs, closest_valid_lat_key, closest_valid_long_key)
    ports_chlorophyl_january.append(port_chlorophyl_january)
    port_chlorophyl_april = get_temprature_or_chlorophyl_value(chlorophyl_april_dfs, closest_valid_lat_key, closest_valid_long_key)
    ports_chlorophyl_april.append(port_chlorophyl_april)
    port_chlorophyl_july = get_temprature_or_chlorophyl_value(chlorophyl_july_dfs, closest_valid_lat_key, closest_valid_long_key)
    ports_chlorophyl_july.append(port_chlorophyl_july)
    port_chlorophyl_october = get_temprature_or_chlorophyl_value(chlorophyl_october_dfs, closest_valid_lat_key, closest_valid_long_key)
    ports_chlorophyl_october.append(port_chlorophyl_october)

ports_in_routes_df['NASA Temp Jan'] = ports_temprature_january
ports_in_routes_df['NASA Temp Apr'] = ports_temprature_april
ports_in_routes_df['NASA Temp Jul'] = ports_temprature_july
ports_in_routes_df['NASA Temp Oct'] = ports_temprature_october

ports_in_routes_df['NASA Chlor Jan'] = ports_chlorophyl_january
ports_in_routes_df['NASA Chlor Apr'] = ports_chlorophyl_april
ports_in_routes_df['NASA Chlor Jul'] = ports_chlorophyl_july
ports_in_routes_df['NASA Chlor Oct'] = ports_chlorophyl_october

In [140]:
PORTS_CONDITIONS_PATH = r"C:\Users\yairs\OneDrive\Documents\University\Master\Ships\datasets from Doron\environment conditions\Ports dataset with their temprature and salinity.csv"
ports_conditions_df = pd.read_csv(PORTS_CONDITIONS_PATH)
ports_conditions_df = ports_conditions_df.loc[ports_conditions_df['PortName'].str.lower().isin(ports_in_routes_df)]
ports_conditions_df = ports_conditions_df[['PortName', 'MinTemp', 'MaxTemp', 'AnnualTemp', 'Salinity']]
print(f"The ports conditions dataset contains {len(ports_conditions_df)} relevant ports")

ports_min_temp = []
ports_max_temp = []
ports_annual_temp = []
ports_salinity = []

The ports conditions dataset contains 85 relevant ports


In [ ]:
OUTPUT_PORTS_DATASET_PATH = r"C:\Users\yairs\OneDrive\Documents\University\Master\Ships\datasets from Doron\environment conditions\final_ports_dataset.csv"
ports_in_routes_df.to_csv(OUTPUT_PORTS_DATASET_PATH, index=False)

In [ ]:
SALINITY_JANUARY_PATH = r"C:\Users\yairs\OneDrive\Documents\University\Master\Ships\datasets from Doron\environment conditions\salinity data from CEDA 2018\jan_sal_good.csv"
SALINITY_APRIL_PATH = r"C:\Users\yairs\OneDrive\Documents\University\Master\Ships\datasets from Doron\environment conditions\salinity data from CEDA 2018\apr_sal_good.csv"
SALINITY_JULY_PATH = r"C:\Users\yairs\OneDrive\Documents\University\Master\Ships\datasets from Doron\environment conditions\salinity data from CEDA 2018\jul_sal_good.csv"
SALINITY_OCTOBER_PATH = r"C:\Users\yairs\OneDrive\Documents\University\Master\Ships\datasets from Doron\environment conditions\salinity data from CEDA 2018\oct_sal_good.csv"